In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json

from config import api_key
import pprint

import datetime as dt
from datetime import date, timedelta

In [2]:
url="http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31"
url

'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31'

In [3]:
response=requests.get(url)
response

<Response [200]>

In [4]:
weather_json=response.json()
weather_json


{'data': {'request': [{'type': 'City', 'query': 'New York'}],
  'weather': [{'date': '2019-01-01',
    'astronomy': [{'sunrise': '07:20 AM',
      'sunset': '04:39 PM',
      'moonrise': '02:59 AM',
      'moonset': '01:47 PM',
      'moon_phase': 'Waning Crescent',
      'moon_illumination': '26'}],
    'maxtempC': '12',
    'maxtempF': '53',
    'mintempC': '-0',
    'mintempF': '32',
    'avgtempC': '10',
    'avgtempF': '50',
    'totalSnow_cm': '0.0',
    'sunHour': '6.9',
    'uvIndex': '3',
    'hourly': [{'time': '0',
      'tempC': '-0',
      'tempF': '32',
      'windspeedMiles': '10',
      'windspeedKmph': '15',
      'winddirDegree': '163',
      'winddir16Point': 'SSE',
      'weatherCode': '308',
      'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0034_cloudy_with_heavy_rain_night.png'}],
      'weatherDesc': [{'value': 'Heavy rain'}],
      'precipMM': '13.9',
      'precipInches': '0.5',
      'humidity': '89',
      

In [5]:
weather_json['data']['weather'][0]['date']

'2019-01-01'

In [6]:
weather_json['data']['weather'][0]['maxtempC']

'12'

In [23]:
weather_json['data']['weather'][0]['hourly'][0]['humidity']

'89'

In [ ]:
weather_json['data']['weather'][0]['hourly'][7]['windspeedKmph']

In [25]:
# day 1
# daily avg windspeed in Kmph for hour=0 to 7
# daily total precipitation in MM for hour=0 to 7
# daily average cloud cover
windspeed = 0
total_precip = 0
cloud = 0
x = len(weather_json['data']['weather'][0]['hourly'])
for i in range(x):
    windspeed += int(weather_json['data']['weather'][0]['hourly'][i]['windspeedKmph'])
    total_precip += float(weather_json['data']['weather'][0]['hourly'][i]['precipMM'])
    cloud += int(weather_json['data']['weather'][0]['hourly'][i]['cloudcover'])
    humidity = int(weather_json['data']['weather'][0]['hourly'][i]['humidity'])

avg_windspeed = windspeed/x
avg_humidity = humidity/x
avg_cloud = cloud/x
print(avg_windspeed)
print(total_precip)
print(avg_cloud)
print(avg_humidity)

16.625
26.9
46.75
7.625


In [ ]:
url="http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31"
response=requests.get(url)

weather_json=response.json()
weather_json


In [15]:
# request one year of data
start_date = '2019-01-01'
end_date = '2019-12-31'
# Use the start and end date to create a range of dates
start_dates = pd.date_range(start_date, end_date, freq='MS')
start_dates = list(np.ravel(pd.to_datetime(start_dates).strftime("%Y-%m-%d")))

end_dates = pd.date_range(end = end_date, periods = 12, freq='M')
end_dates = list(np.ravel(pd.to_datetime(end_dates).strftime("%Y-%m-%d")))


In [16]:
start_dates

['2019-01-01',
 '2019-02-01',
 '2019-03-01',
 '2019-04-01',
 '2019-05-01',
 '2019-06-01',
 '2019-07-01',
 '2019-08-01',
 '2019-09-01',
 '2019-10-01',
 '2019-11-01',
 '2019-12-01']

In [17]:
end_dates

['2019-01-31',
 '2019-02-28',
 '2019-03-31',
 '2019-04-30',
 '2019-05-31',
 '2019-06-30',
 '2019-07-31',
 '2019-08-31',
 '2019-09-30',
 '2019-10-31',
 '2019-11-30',
 '2019-12-31']

In [ ]:
#url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date={start_dates[0]}&enddate={end_dates[0]}"
url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key="
dates = f"&q=New+York&format=json&date={start_dates[0]}&enddate={end_dates[0]}"
query_url = url + {api_key} + dates

response=requests.get(url)

weather_json=response.json()
# weather_json

In [18]:
len(end_dates)

12

In [32]:
# loop thru all months in 2019
nyc = 'New+York'
url = f"http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key="
city = f"&q={nyc}&format=json&date="

for dt in range(len(start_dates)):
    query_url = f"{url}{api_key}{city}{start_dates[dt]}&enddate={end_dates[dt]}"
    print(query_url)
    
# request for data
    response=requests.get(query_url)
    weather_json=response.json()

# loop through one month data
    days = len(weather_json['data']['weather'])
    for day in range(days):
        dates = weather_json['data']['weather'][day]['date'] # date
        maxtempC = weather_json['data']['weather'][day]['maxtempC'] # maxTempC

        windspeed = 0
        total_precip = 0
        cloud = 0
        x = len(weather_json['data']['weather'][day]['hourly'])
        for i in range(x):
            windspeed += int(weather_json['data']['weather'][day]['hourly'][i]['windspeedKmph'])
            total_precip += float(weather_json['data']['weather'][day]['hourly'][i]['precipMM'])
            cloud += int(weather_json['data']['weather'][day]['hourly'][i]['cloudcover'])
            humidity += int(weather_json['data']['weather'][day]['hourly'][i]['humidity'])

        avg_windspeed = windspeed/x
        avg_humidity = humidity/x
        avg_cloud = cloud/x

        print(f"{dates} {maxtempC} {avg_windspeed} {avg_cloud} {total_precip} {avg_humidity}")

http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=9d847c37070c4ddd88a31949201512&q=New+York&format=json&date=2019-01-01&enddate=2019-01-31
2019-01-01 12 16.625 46.75 26.9 3873.5
2019-01-02 4 10.0 10.625 0.0 3928.75
2019-01-03 5 9.75 44.875 1.0 3993.375
2019-01-04 7 13.125 32.5 0.0 4064.125
2019-01-05 6 12.25 87.125 27.800000000000004 4135.25
2019-01-06 7 14.375 27.125 0.0 4197.5
2019-01-07 3 7.75 40.875 0.0 4250.375
2019-01-08 7 10.0 96.5 4.3999999999999995 4333.125
2019-01-09 6 20.625 52.375 0.30000000000000004 4401.125
2019-01-10 2 18.625 58.125 0.2 4458.875
2019-01-11 -2 15.75 7.875 0.0 4504.0
2019-01-12 0 6.875 36.0 0.0 4556.125
2019-01-13 -1 10.375 78.5 0.4 4609.625
2019-01-14 -1 11.75 31.875 0.0 4659.5
2019-01-15 3 8.0 25.0 0.0 4712.625
2019-01-16 4 10.25 31.75 0.0 4776.25
2019-01-17 -1 9.625 54.75 0.0 4818.875
2019-01-18 4 6.25 81.0 0.6 4890.25
2019-01-19 2 9.0 56.125 1.8 4960.25
2019-01-20 8 21.375 89.125 56.0 5041.875
2019-01-21 -12 26.75 55.875 0.2 5095.5
201

2019-07-01 32 11.75 7.125 0.0 14225.75
2019-07-02 33 14.875 35.5 0.2 14273.75
2019-07-03 30 7.75 73.875 5.3999999999999995 14334.0
2019-07-04 31 11.25 10.125 0.0 14390.25
2019-07-05 29 12.875 37.25 2.4 14460.125
2019-07-06 33 13.625 54.125 10.6 14523.375
2019-07-07 29 14.375 54.875 1.2000000000000002 14582.375
2019-07-08 27 7.625 61.0 2.9999999999999996 14642.875
2019-07-09 36 8.375 7.125 0.0 14692.5
2019-07-10 33 10.375 10.875 0.0 14744.25
2019-07-11 29 18.0 45.875 20.5 14806.125
2019-07-12 31 15.25 51.625 22.500000000000004 14866.125
2019-07-13 33 13.75 5.75 0.0 14916.375
2019-07-14 34 15.625 8.375 0.0 14962.125
2019-07-15 34 11.25 4.875 0.0 15002.5
2019-07-16 33 10.0 12.125 0.0 15046.125
2019-07-17 35 12.5 79.375 19.5 15102.625
2019-07-18 29 9.25 82.625 24.599999999999998 15172.375
2019-07-19 35 11.625 30.25 0.2 15234.75
2019-07-20 37 15.25 46.5 6.9 15287.125
2019-07-21 38 15.625 53.625 1.5999999999999999 15336.375
2019-07-22 31 11.0 70.75 45.8 15394.0
2019-07-23 25 13.25 89.5 39.9 

In [29]:
# loop through one month data
days = len(weather_json['data']['weather'])
for day in range(days):
    dates = weather_json['data']['weather'][day]['date'] # date
    maxtempC = weather_json['data']['weather'][day]['maxtempC'] # maxTempC
    
    windspeed = 0
    total_precip = 0
    cloud = 0
    x = len(weather_json['data']['weather'][day]['hourly'])
    for i in range(x):
        windspeed += int(weather_json['data']['weather'][day]['hourly'][i]['windspeedKmph'])
        total_precip += float(weather_json['data']['weather'][day]['hourly'][i]['precipMM'])
        cloud += int(weather_json['data']['weather'][day]['hourly'][i]['cloudcover'])
        humidity += int(weather_json['data']['weather'][day]['hourly'][i]['humidity'])
    
    avg_windspeed = windspeed/x
    avg_humidity = humidity/x
    avg_cloud = cloud/x
    
    print(f"{dates} {maxtempC} {avg_windspeed} {avg_cloud} {total_precip} {avg_humidity}")

2019-01-01 12 16.625 46.75 26.9 1978.625
2019-01-02 4 10.0 10.625 0.0 2033.875
2019-01-03 5 9.75 44.875 1.0 2098.5
2019-01-04 7 13.125 32.5 0.0 2169.25
2019-01-05 6 12.25 87.125 27.800000000000004 2240.375
2019-01-06 7 14.375 27.125 0.0 2302.625
2019-01-07 3 7.75 40.875 0.0 2355.5
2019-01-08 7 10.0 96.5 4.3999999999999995 2438.25
2019-01-09 6 20.625 52.375 0.30000000000000004 2506.25
2019-01-10 2 18.625 58.125 0.2 2564.0
2019-01-11 -2 15.75 7.875 0.0 2609.125
2019-01-12 0 6.875 36.0 0.0 2661.25
2019-01-13 -1 10.375 78.5 0.4 2714.75
2019-01-14 -1 11.75 31.875 0.0 2764.625
2019-01-15 3 8.0 25.0 0.0 2817.75
2019-01-16 4 10.25 31.75 0.0 2881.375
2019-01-17 -1 9.625 54.75 0.0 2924.0
2019-01-18 4 6.25 81.0 0.6 2995.375
2019-01-19 2 9.0 56.125 1.8 3065.375
2019-01-20 8 21.375 89.125 56.0 3147.0
2019-01-21 -12 26.75 55.875 0.2 3200.625
2019-01-22 -3 9.75 10.875 0.0 3250.75
2019-01-23 5 13.75 73.5 0.2 3322.625
2019-01-24 10 17.875 92.0 45.300000000000004 3409.0
2019-01-25 1 16.5 12.875 0.0 3470

In [ ]:
#loop through dates and perform a request for data on each
#max temp, percipitation, wind speed, cloudiness, lat, long
dates=[]
max_temp=[]
percip=[]
wind_speed=[]
cloudiness=[]
lat=[]
long=[]

for date in dates:
    response=requests.get(url).json()
    date.append(response[''])